# Predicting storage Memory of laptop

In [1]:
import numpy as np 
import pandas as pd

In [4]:
dataset = pd.read_csv('laptop_price_processed.csv')

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   TypeName          1303 non-null   object 
 3   Inches            1303 non-null   float64
 4   Ram               1303 non-null   int64  
 5   OpSys             1303 non-null   object 
 6   Weight            1303 non-null   float64
 7   Price_euros       1303 non-null   float64
 8   IPS_Panel         1303 non-null   int64  
 9   Retina_Display    1303 non-null   int64  
 10  Touchscreen       1303 non-null   int64  
 11  Resolution_X      1303 non-null   int64  
 12  Resolution_Y      1303 non-null   int64  
 13  Total_Pixels      1303 non-null   int64  
 14  High_Resolution   1303 non-null   int64  
 15  Product_Series    1303 non-null   object 
 16  Cpu_Brand         1303 non-null   object 


### encoding categorical column

In [144]:
from sklearn.preprocessing import LabelEncoder
categorical_col = ['TypeName', 'OpSys', 'Cpu_Series', 'Gpu_Series', 'Memory_Type']

le = LabelEncoder()

for col in categorical_col:
  dataset[col] = le.fit_transform(dataset[col])

In [145]:
X = dataset[['TypeName', 'OpSys', 'Cpu_Series', 'Gpu_Series', 'Memory_Type', 'Price_euros', 'Total_Pixels']]
y = dataset['Memory_Size(GB)']

### spliting with training and testing data

In [146]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

### scaling data

In [147]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### linear regression

In [148]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [149]:
y_pred = lr.predict(X_test)

In [150]:
from sklearn.metrics import r2_score
score = r2_score(y_test, y_pred)
print(f'r2 Score {score}')

r2 Score 0.13244121999522218


### Cat boost

In [151]:
from catboost import CatBoostClassifier
catb = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass', random_state=21)
catb.fit(X_train, y_train)

0:	learn: 2.5377531	total: 20.3ms	remaining: 2.01s
1:	learn: 2.2289701	total: 38.7ms	remaining: 1.9s
2:	learn: 1.9880571	total: 56.8ms	remaining: 1.83s
3:	learn: 1.8141527	total: 74.8ms	remaining: 1.79s
4:	learn: 1.6813762	total: 96.5ms	remaining: 1.83s
5:	learn: 1.5819855	total: 115ms	remaining: 1.8s
6:	learn: 1.4964080	total: 134ms	remaining: 1.78s
7:	learn: 1.4276184	total: 151ms	remaining: 1.74s
8:	learn: 1.3705561	total: 174ms	remaining: 1.76s
9:	learn: 1.3176056	total: 190ms	remaining: 1.71s
10:	learn: 1.2754672	total: 211ms	remaining: 1.71s
11:	learn: 1.2400170	total: 235ms	remaining: 1.72s
12:	learn: 1.2100664	total: 259ms	remaining: 1.73s
13:	learn: 1.1756425	total: 283ms	remaining: 1.74s
14:	learn: 1.1473415	total: 307ms	remaining: 1.74s
15:	learn: 1.1240516	total: 334ms	remaining: 1.75s
16:	learn: 1.0975153	total: 358ms	remaining: 1.75s
17:	learn: 1.0798525	total: 382ms	remaining: 1.74s
18:	learn: 1.0634328	total: 404ms	remaining: 1.72s
19:	learn: 1.0455195	total: 424ms	rema

In [152]:
y_pred2 = catb.predict(X_test)

In [153]:
from sklearn.metrics import r2_score
score2 = r2_score(y_test, y_pred2)
print(f'r2 Score {score2}')

r2 Score 0.6717314748522782


### extratrees

In [154]:
from sklearn.ensemble import ExtraTreesClassifier
etreec = ExtraTreesClassifier(n_estimators=100, max_depth=10, random_state=42, min_samples_split=5, min_samples_leaf=3)
etreec.fit(X_train, y_train)

ExtraTreesClassifier(max_depth=10, min_samples_leaf=3, min_samples_split=5,
                     random_state=42)

In [155]:
y_pred3 = etreec.predict(X_test)

In [156]:
from sklearn.metrics import r2_score
score3 = r2_score(y_test, y_pred3)
print(f'r2 Score {score3}')

r2 Score 0.5712892617029134


### stacking

In [157]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Define base models (level 0)
base_learners = [
    ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('lr', LogisticRegression(random_state=42))
]

# Define the meta-model (level 1)
meta_model = LogisticRegression()

In [158]:
from sklearn.ensemble import StackingClassifier
stackc = StackingClassifier(estimators=base_learners, final_estimator=meta_model)
stackc.fit(X_train, y_train)

c:\Users\dabhi\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\dabhi\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1426: RuntimeWarning: Number of classes in training fold (20) does not match total number of classes (21). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
c:\Users\dabhi\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1426: RuntimeWarning: Number of classes in training fold (19) does not match total number of classes (21). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
c:\Users\dabhi\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1426: RuntimeWarning: Number of classes in

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=50,
                                                       random_state=42)),
                               ('knn', KNeighborsClassifier()),
                               ('lr', LogisticRegression(random_state=42))],
                   final_estimator=LogisticRegression())

In [159]:
y_pred4 = stackc.predict(X_test)

In [160]:
from sklearn.metrics import r2_score
score4 = r2_score(y_test, y_pred4)
print(f'r2 Score {score4}')

r2 Score 0.6260884838743361


### randomforest

In [161]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=57, random_state=21)
rfc.fit(X_train, y_train)

RandomForestClassifier(n_estimators=57, random_state=21)

In [162]:
y_pred5 = rfc.predict(X_test)

In [163]:
from sklearn.metrics import r2_score
score5 = r2_score(y_test, y_pred5)
print(f'r2 Score {score5}')

r2 Score 0.4622443390538137


In [164]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=57, random_state=21, max_depth=7)
rfr.fit(X_train, y_train)

RandomForestRegressor(max_depth=7, n_estimators=57, random_state=21)

In [165]:
y_pred6 = rfr.predict(X_test)

In [166]:
from sklearn.metrics import r2_score
score6 = r2_score(y_test, y_pred6)
print(f'r2 Score {score6}')

r2 Score 0.7343244705215


### logistic regression

In [167]:
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression(random_state=21)
logr.fit(X_train, y_train)

LogisticRegression(random_state=21)

In [168]:
y_pred7 = logr.predict(X_test)

In [169]:
from sklearn.metrics import r2_score
score7 = r2_score(y_test, y_pred7)
print(f'r2 Score {score7}')

r2 Score 0.5563059723468355


In [170]:
from xgboost import XGBRegressor
xgbr = XGBRegressor(n_estimators=15, max_depth=4, eta=0.1)
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=15,
             n_jobs=None, num_parallel_tree=None, ...)

In [171]:
y_pred8 = xgbr.predict(X_test)

In [172]:
from sklearn.metrics import r2_score
score8 = r2_score(y_test, y_pred8)
print(f'r2 Score {score8}')

r2 Score 0.6906700834400992


### SVR

In [173]:
from sklearn.svm import SVR
svr = SVR(kernel = 'rbf')
svr.fit(X, y)

SVR()

In [174]:
y_pred9 = svr.predict(X_test)

c:\Users\dabhi\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


In [175]:
from sklearn.metrics import r2_score
score9 = r2_score(y_test, y_pred9)
print(f'r2 Score {score9}')

r2 Score -0.0829485920605697
